In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [113]:
MAX_PRIORITY = 2 # TODO : read that value in the file directly
RATE = 1e9 / 8 # rate of transmission in the links
OVERHEAD = 42 # bytes

# PROTOTYPE

### Objectives

- Extract data from csv files to put in an appropriate data structure
- Build a network and find shortest path

In [114]:
streams = pd.read_csv("test_cases/small-streams.csv",
                      names=['PCP','StreamName','StreamType','SourceNode','DestinationNode','Size','Period','Deadline']
                     )
topology_cols = [str(i) for i in range(7)]
topology = pd.read_csv("test_cases/small-topology.csv",names=topology_cols).groupby('0')

In [115]:
streams.head(10)

,PCP,StreamName,StreamType,SourceNode,DestinationNode,Size,Period,Deadline
0,0,Flow_0,ATS,node0_0_0_0,node0_0_6_0,80,20000,12049
1,2,Flow_1,ATS,node0_0_4_1,node0_0_3_1,130,2000,13396
2,1,Flow_10,ATS,node0_0_2_1,node0_0_6_1,329,20000,12910
3,0,Flow_11,ATS,node0_0_5_1,node0_0_6_0,373,4000,19044
4,0,Flow_12,ATS,node0_0_0_0,node0_0_6_1,359,4000,17020
5,2,Flow_13,ATS,node0_0_4_1,node0_0_6_0,354,2000,17088
6,1,Flow_14,ATS,node0_0_4_1,node0_0_3_0,467,4000,16154
7,0,Flow_15,ATS,node0_0_4_1,node0_0_1_0,395,20000,15512
8,2,Flow_16,ATS,node0_0_1_1,node0_0_6_0,278,2000,14692
9,0,Flow_17,ATS,node0_0_0_0,node0_0_6_1,112,2000,15756


In [ ]:
"""Extracting the data from csv files to DataFrames"""

switches = topology.get_group('SW')
switches = switches.drop(columns=['4','5','6'])
switches.columns = ['DeviceType','DeviceName','Ports','Domain']

end_systems = topology.get_group('ES')
end_systems = end_systems.drop(columns=['4','5','6'])
end_systems.columns = ['DeviceType','DeviceName','Ports','Domain']

links = topology.get_group('LINK')
links.columns = ['LINK','LinkID','SourceDevice','SourcePort','DestinationDevice','DestinationPort','Domain']

In [117]:
end_systems.head()

,DeviceType,DeviceName,Ports,Domain
8,ES,node0_0_0_0,1,NaN
9,ES,node0_0_0_1,1,NaN
10,ES,node0_0_3_0,1,NaN
11,ES,node0_0_3_1,1,NaN
12,ES,node0_0_5_0,1,NaN


In [118]:
switches.head(20)

,DeviceType,DeviceName,Ports,Domain
0,SW,sw_0_0,8,NaN
1,SW,sw_0_3,8,NaN
2,SW,sw_0_5,8,NaN
3,SW,sw_0_1,8,NaN
4,SW,sw_0_4,8,NaN
5,SW,sw_0_6,8,NaN
6,SW,sw_0_2,8,NaN
7,SW,sw_0_7,8,NaN


In [119]:
links.head(20)

,LINK,LinkID,SourceDevice,SourcePort,DestinationDevice,DestinationPort,Domain
22,LINK,e1,sw_0_0,0.0,sw_0_3,0.0,NaN
23,LINK,e2,sw_0_0,1.0,sw_0_5,0.0,NaN
24,LINK,e3,sw_0_0,2.0,node0_0_0_0,1.0,NaN
25,LINK,e4,sw_0_0,3.0,node0_0_0_1,1.0,NaN
26,LINK,e5,sw_0_3,1.0,sw_0_2,0.0,NaN
27,LINK,e6,sw_0_3,2.0,sw_0_6,0.0,NaN
28,LINK,e7,sw_0_3,3.0,sw_0_7,0.0,NaN
29,LINK,e8,sw_0_3,4.0,node0_0_3_0,1.0,NaN
30,LINK,e9,sw_0_3,5.0,node0_0_3_1,1.0,NaN
31,LINK,e10,sw_0_5,1.0,sw_0_2,1.0,NaN


In [ ]:
class Flow():
    def __init__(self, data_row):
        # filling a class for each stream to manipulate the data more easily
        self.priority = data_row["PCP"]
        self.src = data_row["SourceNode"]
        self.dest = data_row["DestinationNode"]
        self.b = data_row["Size"]
        self.r = data_row["Size"] / data_row["Period"]
        self.deadline = data_row["Deadline"]
        self.name = data_row["StreamName"]
        self.l = data_row["Size"] # packet length
        self.total_delay = None
    
    def find_path(self, G: nx.MultiGraph):
        # first we find the shortest path from the source to the destination of the stream 
        path = nx.shortest_path(G, self.src, self.dest) 
        self.path = path
        graph = nx.path_graph(path)
        edges = graph.edges()
        self.links = []
        for edge in edges:
            link = G.edges.get([edge[0], edge[1], 0])
            self.links.append({"edges": [G.nodes[edge[0]], G.nodes[edge[1]]], "data": link})

    def __repr__(self):
        # just to have a print for comparison with solution.csv file
        path = "->".join(self.path)
        return f"{self.name},{round(self.total_delay * 1e6, 3)},{self.deadline},{path}"

    def fill_nodes(self, G):
        # takes each nodes and adds this flow to the array
        # of all flows going through
        for link in self.links:
            [edge0, edge1] = link["edges"]
            link_id = link["data"]["link_id"]
            edge0["output_flows"][self.priority] = [
                *edge0["output_flows"].get(self.priority,[]),
                self
            ] # creates an array if it does not exist
            edge1["input_flows"][self.priority] = edge1["input_flows"].get(self.priority, dict())
            # creates a dictionary if it does not exist
            edge1["input_flows"][self.priority][link_id] = [
                *edge1["input_flows"][self.priority].get(link_id, []),
                self
            ] # creates an array for each link if it does not exist
    def hop_delay(self, link):
        [edge0, edge1] = link["edges"]
        output_flows = edge0["output_flows"]
        input_flows = edge1["input_flows"]
        link_id = link["data"]["link_id"]

        # higher priority 
        bH = 0
        rH = 0
        for priority in range(self.priority+1, MAX_PRIORITY+1):
            for flow in output_flows.get(priority, []):
                bH += flow.b
                rH += flow.r

        # lower priority
        lL = 0 
        for priority in range(0, self.priority):
            for flow in output_flows.get(priority, []):
                if flow.l > lL:
                    lL = flow.l

        max_delay = 0
        for flow in input_flows[self.priority][link_id]:
            bc = 0
            for bc_flow in input_flows[self.priority][link_id]:
                if bc_flow != flow:
                    bc += bc_flow.b # we add every flow's burst except for j to bc
            lj = flow.l
            delay = (bH + bc + lL) / (RATE - rH) + lj / RATE + OVERHEAD / RATE
            if delay > max_delay:
                max_delay = delay
        
        return max_delay

    def get_total_delay(self):
        total_delay = 0
        for link in self.links:
            total_delay += self.hop_delay(link)
        self.total_delay = total_delay



In [152]:
G = nx.MultiGraph()
# multi graph to allow multiple links between two same nodes

# this creates the links as well as the necessary nodes
for link in links.iterrows():
    source = link[1]['SourceDevice']
    destination = link[1]['DestinationDevice']
    source_port = link[1]['SourcePort']
    link_id = link[1]['LinkID']
    destination_port = link[1]['DestinationPort']
    G.add_edge(source,
               destination,
               source_port=source_port,
               destination_port=destination_port,
               link_id=link_id,
               flows=dict()
              )

for end_system in end_systems.iterrows():
    # the names are supposedly already inside the graph because of the link creation
    name = end_system[1]['DeviceName']
    G.nodes[name]['ports'] = end_system[1]['Ports']
    G.nodes[name]['input_flows'] = dict()
    G.nodes[name]['output_flows'] = dict()

for switch in switches.iterrows():
    name = switch[1]['DeviceName']
    G.nodes[name]['ports'] = switch[1]['Ports']
    G.nodes[name]['input_flows'] = dict()
    G.nodes[name]['output_flows'] = dict()

In [153]:
flows = []
for stream in streams.iterrows():
    flow = Flow(stream[1])
    flow.find_path(G)
    flows.append(flow)

for flow in flows:
    flow.fill_nodes(G)
    pass

In [155]:
for flow in flows:
    flow.get_total_delay()
    # print(f"{flow.name}, {flow.total_delay * 1e6}")
    print(flow)

Flow_0,96.568,12049,node0_0_0_0->sw_0_0->sw_0_3->sw_0_6->node0_0_6_0
Flow_1,43.24,13396,node0_0_4_1->sw_0_4->sw_0_6->sw_0_3->node0_0_3_1
Flow_10,53.104,12910,node0_0_2_1->sw_0_2->sw_0_3->sw_0_6->node0_0_6_1
Flow_11,105.024,19044,node0_0_5_1->sw_0_5->sw_0_0->sw_0_3->sw_0_6->node0_0_6_0
Flow_12,98.928,17020,node0_0_0_0->sw_0_0->sw_0_3->sw_0_6->node0_0_6_1
Flow_13,36.88,17088,node0_0_4_1->sw_0_4->sw_0_6->node0_0_6_0
Flow_14,91.392,16154,node0_0_4_1->sw_0_4->sw_0_6->sw_0_3->node0_0_3_0
Flow_15,57.64,15512,node0_0_4_1->sw_0_4->sw_0_1->node0_0_1_0
Flow_16,24.112,14692,node0_0_1_1->sw_0_1->sw_0_6->node0_0_6_0
Flow_17,98.928,15756,node0_0_0_0->sw_0_0->sw_0_3->sw_0_6->node0_0_6_1
Flow_18,24.624,14909,node0_0_2_1->sw_0_2->sw_0_3->node0_0_3_0
Flow_19,131.648,13974,node0_0_4_0->sw_0_4->sw_0_6->sw_0_3->sw_0_0->node0_0_0_1
Flow_2,24.112,19752,node0_0_1_1->sw_0_1->sw_0_6->node0_0_6_0
Flow_20,98.928,18037,node0_0_0_0->sw_0_0->sw_0_3->sw_0_6->node0_0_6_1
Flow_21,52.616,13328,node0_0_4_1->sw_0_4->sw_0_6

In [156]:
for k, item in G.edges.items():
    print(item)

{'source_port': 0.0, 'destination_port': 0.0, 'link_id': 'e1', 'flows': {}}
{'source_port': 1.0, 'destination_port': 0.0, 'link_id': 'e2', 'flows': {}}
{'source_port': 2.0, 'destination_port': 1.0, 'link_id': 'e3', 'flows': {}}
{'source_port': 3.0, 'destination_port': 1.0, 'link_id': 'e4', 'flows': {}}
{'source_port': 1.0, 'destination_port': 0.0, 'link_id': 'e5', 'flows': {}}
{'source_port': 2.0, 'destination_port': 0.0, 'link_id': 'e6', 'flows': {}}
{'source_port': 3.0, 'destination_port': 0.0, 'link_id': 'e7', 'flows': {}}
{'source_port': 4.0, 'destination_port': 1.0, 'link_id': 'e8', 'flows': {}}
{'source_port': 5.0, 'destination_port': 1.0, 'link_id': 'e9', 'flows': {}}
{'source_port': 1.0, 'destination_port': 1.0, 'link_id': 'e10', 'flows': {}}
{'source_port': 2.0, 'destination_port': 1.0, 'link_id': 'e11', 'flows': {}}
{'source_port': 3.0, 'destination_port': 1.0, 'link_id': 'e12', 'flows': {}}
{'source_port': 2.0, 'destination_port': 1.0, 'link_id': 'e23', 'flows': {}}
{'source